 Author: Shen Chan Huang
 
 For: Dr. Mingon Kang 
 
 CS622 -- Assignment 2

In [183]:
#Read MNIST Train and Test Data
import pandas as pd
import numpy as np
import statistics as st

#merge two files together using concat
csv_to_merge = ['MNIST_training.csv', 'MNIST_test.csv']
X_full = pd.concat([pd.read_csv(f) for f in csv_to_merge], ignore_index=True)

In [192]:
#Writing splits as function, input N fold and experiment number
def N_Fold_CV(N,exp_num,samp_range):
    #set range of sampling
    
    #select samples for training and testing 
    X_test = X_full.iloc[samp_range*(exp_num-1):samp_range*exp_num]
    labels_to_drop = list(X_test.index.values)
    X_train = X_full.drop(labels_to_drop)
    return X_train, X_test

In [185]:
#Minkowski distance, input vectors and power. 
def Mink_Dist(a,b,r):
    D = np.power(np.abs(a-b),r)
    x = sum(D)
    return np.power(x, 1/r)

In [186]:
#Cosine Similarity, input vectors
def cos_sim(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [187]:
#KNN function returns the accuracy. It calculates the distance for 1 test sample at a time and predicts its class.
def KNN(r,K):
    vote_count = 0
    for i in range(0,len(X_test)):
        D_array=[]
        for j in range(0,len(X_train)):
            a = X_train[j][1:]
            b = X_test[i][1:]
            #Dist = Mink_Dist(a,b,r) #Calculate the distance between the query-instance and all the training examples
            Dist = cos_sim(a,b) #r is not used for cos similarity
            D_array.append([X_train[j][0],Dist])
    #Sort the distance and determine nearest neighbors based on the k-th minimum distance
        D_array = sorted(D_array, key=lambda x: x[1], reverse=True) #set False for Minkowski, True for cos similarity
        majority = st.mode(p[0] for p in D_array[:K]) #Gather the category Y of nearest neighbors and majority vote
        if majority == X_test[i][0]:
            vote_count += 1
    acc = vote_count/len(X_test)
    return acc

In [205]:
#r is the power in the Minkowski Distance
#K is the number of nearest neighbors
#N is the number of folds in cross validation
r = 2
K = 10
N = 5
res = []
samp_range = int(round(len(X_full)/N,0))
#shuffle the dataframe since it is ordered
X_full = X_full.sample(frac=1)

for i in range(N):
    #Set N fold split
    X_train, X_test = N_Fold_CV(N,i+1,samp_range)
    #converting to numpy arrays for easier calculations
    X_train = X_train.to_numpy()
    X_test = X_test.to_numpy()
    res.append(KNN(r,K))
avg_acc = np.mean(res)
 

In [203]:
space = ' '
print(f'Experiment {space*5} Accuracy')
for j in range(N):
    print(f'Experiment {j+1} {space*4} {res[j]}' )
print(f'Average {space*9} {avg_acc}')

Experiment       Accuracy
Experiment 1      0.93
Experiment 2      0.88
Experiment 3      0.88
Experiment 4      0.88
Experiment 5      0.89
Experiment 6      0.82
Experiment 7      0.89
Experiment 8      0.93
Experiment 9      0.84
Experiment 10      0.9090909090909091
Average           0.884909090909091
